In [1]:
print(1+1)

2


In [2]:
import numpy

In [3]:
## MCTS TIc TAc Toe

In [2]:
import random
import math
from copy import deepcopy

class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.wins = 0

    def is_fully_expanded(self):
        return len(self.children) == len(self.state.get_legal_actions())

    def best_child(self, c_param=1.4):
        choices_weights = [
            (child.wins / child.visits) + c_param * math.sqrt((2 * math.log(self.visits) / child.visits))
            for child in self.children
        ]
        return self.children[choices_weights.index(max(choices_weights))]

class TicTacToe:
    def __init__(self):
        self.board = [[None, None, None], [None, None, None], [None, None, None]]
        self.current_player = 'X'

    def clone(self):
        clone = TicTacToe()
        clone.board = deepcopy(self.board)
        clone.current_player = self.current_player
        return clone

    def get_legal_actions(self):
        actions = []
        for r in range(3):
            for c in range(3):
                if self.board[r][c] is None:
                    actions.append((r, c))
        return actions

    def play_action(self, action):
        r, c = action
        if self.board[r][c] is None:
            self.board[r][c] = self.current_player
            self.current_player = 'O' if self.current_player == 'X' else 'X'

    def is_terminal(self):
        return self.get_winner() is not None or all(cell is not None for row in self.board for cell in row)

    def get_winner(self):
        for line in self.board + list(map(list, zip(*self.board))) + \
                    [[self.board[i][i] for i in range(3)], [self.board[i][2-i] for i in range(3)]]:
            if line[0] is not None and line[0] == line[1] == line[2]:
                return line[0]
        return None

    def get_result(self, player):
        winner = self.get_winner()
        if winner == player:
            return 1
        elif winner is None:
            return 0.5
        else:
            return 0

def MCTS(root, iterations=1000):
    for _ in range(iterations):
        node = tree_policy(root)
        reward = default_policy(node.state)
        backup(node, reward)
    return root.best_child(0)

def tree_policy(node):
    while not node.state.is_terminal():
        if not node.is_fully_expanded():
            return expand(node)
        else:
            node = node.best_child()
    return node

def expand(node):
    tried_children = [child.state for child in node.children]
    new_state = node.state.clone()
    action = random.choice([act for act in new_state.get_legal_actions() if new_state.clone().play_action(act) not in tried_children])
    new_state.play_action(action)
    child_node = Node(new_state, node)
    node.children.append(child_node)
    return child_node

def default_policy(state):
    while not state.is_terminal():
        state.play_action(random.choice(state.get_legal_actions()))
    return state.get_result(state.current_player)

def backup(node, reward):
    while node is not None:
        node.visits += 1
        node.wins += reward
        node = node.parent
        reward = 1 - reward

def print_board(board):
    for row in board:
        print(' | '.join([cell if cell is not None else ' ' for cell in row]))
        print('-' * 5)

if __name__ == "__main__":
    game = TicTacToe()
    root = Node(game)

    while not game.is_terminal():
        if game.current_player == 'X':
            root = MCTS(root, 1000)
            game = root.state
        else:
            action = random.choice(game.get_legal_actions())
            game.play_action(action)
            root = Node(game)

        print_board(game.board)
        print()


X | O | X
-----
X | X | O
-----
O | X | O
-----

